# Colab-DeblurGANv2

Currently only one picture as input allowed.

Original repo: [TAMU-VITA/DeblurGANv2](https://github.com/TAMU-VITA/DeblurGANv2)

My coalb fork: [styler00dollar/Colab-DeblurGANv2](https://github.com/styler00dollar/Colab-DeblurGANv2)

Simple tutoiral:
Place ```input.png``` in ```Google Drive/Colab-DeblurGANv2``` and run these cells. You can create this folder with colab.

In [ ]:
# check gpu
!nvidia-smi

In [ ]:
#@title Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('Google Drive connected.')

Either create input and output folders manually or with colab and place the files there

In [ ]:
#@title [Optional] Creating empty ```"/content/drive/My Drive/Colab-DeblurGANv2/"``` folder in Google Drive
!mkdir "/content/drive/My Drive/Colab-DeblurGANv2/"

# Deblur with InceptionResNet-v2

In [ ]:
#@title Installing and downloading model
%cd /content/
!git clone https://github.com/styler00dollar/Colab-DeblurGANv2
!pip install fire
!pip install pretrainedmodels
!pip install gdown
%cd /content/Colab-DeblurGANv2/models
!gdown --id 1UXcsRVW-6KF23_TNzxw-xC0SzaMfXOaR

In [ ]:
#@title Changing predict.py
%%writefile /content/Colab-DeblurGANv2/predict.py
import os
from glob import glob
from typing import Optional

import cv2
import numpy as np
import torch
import yaml
from fire import Fire
from tqdm import tqdm

from aug import get_normalize
from models.networks import get_generator
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

class Predictor:
    def __init__(self, weights_path: str, model_name: str = ''):
        with open('config/config.yaml') as cfg_file:
            cfg = yaml.safe_load(cfg_file)
        model = get_generator(model_name or cfg['model'])
        model.load_state_dict(torch.load(weights_path)['model'])
        self.model = model.cuda()
        self.model.train(True)
        # GAN inference should be in train mode to use actual stats in norm layers,
        # it's not a bug
        self.normalize_fn = get_normalize()

    @staticmethod
    def _array_to_batch(x):
        x = np.transpose(x, (2, 0, 1))
        x = np.expand_dims(x, 0)
        return torch.from_numpy(x)

    def _preprocess(self, x: np.ndarray, mask: Optional[np.ndarray]):
        x, _ = self.normalize_fn(x, x)
        if mask is None:
            mask = np.ones_like(x, dtype=np.float32)
        else:
            mask = np.round(mask.astype('float32') / 255)

        h, w, _ = x.shape
        block_size = 32
        min_height = (h // block_size + 1) * block_size
        min_width = (w // block_size + 1) * block_size

        pad_params = {'mode': 'constant',
                      'constant_values': 0,
                      'pad_width': ((0, min_height - h), (0, min_width - w), (0, 0))
                      }
        x = np.pad(x, **pad_params)
        mask = np.pad(mask, **pad_params)

        return map(self._array_to_batch, (x, mask)), h, w

    @staticmethod
    def _postprocess(x: torch.Tensor) -> np.ndarray:
        x, = x
        x = x.detach().cpu().float().numpy()
        x = (np.transpose(x, (1, 2, 0)) + 1) / 2.0 * 255.0
        return x.astype('uint8')

    def __call__(self, img: np.ndarray, mask: Optional[np.ndarray], ignore_mask=True) -> np.ndarray:
        (img, mask), h, w = self._preprocess(img, mask)
        with torch.no_grad():
            inputs = [img.cuda()]
            if not ignore_mask:
                inputs += [mask]
            pred = self.model(*inputs)
        return self._postprocess(pred)[:h, :w, :]


def process_video(pairs, predictor, output_dir):
    for video_filepath, mask in tqdm(pairs):
        video_filename = os.path.basename(video_filepath)
        output_filepath = os.path.join(output_dir, os.path.splitext(video_filename)[0]+'_deblur.mp4')
        video_in = cv2.VideoCapture(video_filepath)
        fps = video_in.get(cv2.CAP_PROP_FPS)
        width = int(video_in.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(video_in.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frame_num = int(video_in.get(cv2.CAP_PROP_FRAME_COUNT))
        video_out = cv2.VideoWriter(output_filepath, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width, height))
        tqdm.write(f'process {video_filepath} to {output_filepath}, {fps}fps, resolution: {width}x{height}')
        for frame_num in tqdm(range(total_frame_num), desc=video_filename):
            res, img = video_in.read()
            if not res:
                break
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            pred = predictor(img, mask)
            pred = cv2.cvtColor(pred, cv2.COLOR_RGB2BGR)
            video_out.write(pred)


def main(img_pattern: str,
         mask_pattern: Optional[str] = None,
         weights_path='/content/Colab-DeblurGANv2/models/fpn_inception.h5',
         out_dir='submit/',
         side_by_side: bool = False,
         video: bool = False):
    def sorted_glob(pattern):
        return sorted(glob(pattern))

    imgs = sorted_glob(img_pattern)
    masks = sorted_glob(mask_pattern) if mask_pattern is not None else [None for _ in imgs]
    pairs = zip(imgs, masks)
    names = sorted([os.path.basename(x) for x in glob(img_pattern)])
    predictor = Predictor(weights_path=weights_path)

    os.makedirs(out_dir, exist_ok=True)
    if not video:
        for name, pair in tqdm(zip(names, pairs), total=len(names)):
            f_img, f_mask = pair
            img, mask = map(cv2.imread, (f_img, f_mask))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            pred = predictor(img, mask)
            if side_by_side:
                pred = np.hstack((img, pred))
            pred = cv2.cvtColor(pred, cv2.COLOR_RGB2BGR)
            cv2.imwrite(os.path.join(out_dir, name),
                        pred)
    else:
        process_video(pairs, predictor, out_dir)


if __name__ == '__main__':
    Fire(main)

In [ ]:
#@title Downloading model from my Google Drive (Very fast download)
!mkdir /root/
!mkdir /root/.cache/
!mkdir /root/.cache/torch/
!mkdir /root/.cache/torch/checkpoints/
%cd /root/.cache/torch/checkpoints/
!gdown --id 1y6GeaoWjhqjRjrXuZvCYEQYlblZGkE6X

In [ ]:
#@title ```[OPTIONAL / ONLY RUN THIS IF YOU HAVE PROBLEMS WITH THE PREVIOUS CELL]``` Testrun to download model file with pretrainedmodels (Download takes around 10 minutes)
%cd /content/
!wget --no-check-certificate https://raw.githubusercontent.com/xinntao/ESRGAN/master/LR/baboon.png
%cd /content/Colab-DeblurGANv2
!python predict.py /content/baboon.png
%cd /content/
!sudo rm /content/baboon.png
!sudo rm /content/Colab-DeblurGANv2/submit/baboon.png

In [ ]:
#@title Run deblur and copy result go Google Drive
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/000in.png"
!cp /content/Colab-DeblurGANv2/submit/000in.png "/content/drive/My Drive/Colab-DeblurGANv2/000out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/001in.png"
!cp /content/Colab-DeblurGANv2/submit/001in.png "/content/drive/My Drive/Colab-DeblurGANv2/001out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/002in.png"
!cp /content/Colab-DeblurGANv2/submit/002in.png "/content/drive/My Drive/Colab-DeblurGANv2/002out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/003in.png"
!cp /content/Colab-DeblurGANv2/submit/003in.png "/content/drive/My Drive/Colab-DeblurGANv2/003out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/004in.png"
!cp /content/Colab-DeblurGANv2/submit/004in.png "/content/drive/My Drive/Colab-DeblurGANv2/004out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/005in.png"
!cp /content/Colab-DeblurGANv2/submit/005in.png "/content/drive/My Drive/Colab-DeblurGANv2/005out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/006in.png"
!cp /content/Colab-DeblurGANv2/submit/006in.png "/content/drive/My Drive/Colab-DeblurGANv2/006out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/007in.png"
!cp /content/Colab-DeblurGANv2/submit/007in.png "/content/drive/My Drive/Colab-DeblurGANv2/007out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/008in.png"
!cp /content/Colab-DeblurGANv2/submit/008in.png "/content/drive/My Drive/Colab-DeblurGANv2/008out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/009in.png"
!cp /content/Colab-DeblurGANv2/submit/009in.png "/content/drive/My Drive/Colab-DeblurGANv2/009out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/010in.png"
!cp /content/Colab-DeblurGANv2/submit/010in.png "/content/drive/My Drive/Colab-DeblurGANv2/010out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/011in.png"
!cp /content/Colab-DeblurGANv2/submit/011in.png "/content/drive/My Drive/Colab-DeblurGANv2/011out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/012in.png"
!cp /content/Colab-DeblurGANv2/submit/012in.png "/content/drive/My Drive/Colab-DeblurGANv2/012out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/013in.png"
!cp /content/Colab-DeblurGANv2/submit/013in.png "/content/drive/My Drive/Colab-DeblurGANv2/013out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/014in.png"
!cp /content/Colab-DeblurGANv2/submit/014in.png "/content/drive/My Drive/Colab-DeblurGANv2/014out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/015in.png"
!cp /content/Colab-DeblurGANv2/submit/015in.png "/content/drive/My Drive/Colab-DeblurGANv2/015out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/016in.png"
!cp /content/Colab-DeblurGANv2/submit/016in.png "/content/drive/My Drive/Colab-DeblurGANv2/016out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/017in.png"
!cp /content/Colab-DeblurGANv2/submit/017in.png "/content/drive/My Drive/Colab-DeblurGANv2/017out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/018in.png"
!cp /content/Colab-DeblurGANv2/submit/018in.png "/content/drive/My Drive/Colab-DeblurGANv2/018out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/019in.png"
!cp /content/Colab-DeblurGANv2/submit/019in.png "/content/drive/My Drive/Colab-DeblurGANv2/019out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/020in.png"
!cp /content/Colab-DeblurGANv2/submit/020in.png "/content/drive/My Drive/Colab-DeblurGANv2/020out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/021in.png"
!cp /content/Colab-DeblurGANv2/submit/021in.png "/content/drive/My Drive/Colab-DeblurGANv2/021out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/022in.png"
!cp /content/Colab-DeblurGANv2/submit/022in.png "/content/drive/My Drive/Colab-DeblurGANv2/022out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/023in.png"
!cp /content/Colab-DeblurGANv2/submit/023in.png "/content/drive/My Drive/Colab-DeblurGANv2/023out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/024in.png"
!cp /content/Colab-DeblurGANv2/submit/024in.png "/content/drive/My Drive/Colab-DeblurGANv2/024out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/025in.png"
!cp /content/Colab-DeblurGANv2/submit/025in.png "/content/drive/My Drive/Colab-DeblurGANv2/025out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/026in.png"
!cp /content/Colab-DeblurGANv2/submit/026in.png "/content/drive/My Drive/Colab-DeblurGANv2/026out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/027in.png"
!cp /content/Colab-DeblurGANv2/submit/027in.png "/content/drive/My Drive/Colab-DeblurGANv2/027out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/028in.png"
!cp /content/Colab-DeblurGANv2/submit/028in.png "/content/drive/My Drive/Colab-DeblurGANv2/028out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/029in.png"
!cp /content/Colab-DeblurGANv2/submit/029in.png "/content/drive/My Drive/Colab-DeblurGANv2/029out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/030in.png"
!cp /content/Colab-DeblurGANv2/submit/030in.png "/content/drive/My Drive/Colab-DeblurGANv2/030out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/031in.png"
!cp /content/Colab-DeblurGANv2/submit/031in.png "/content/drive/My Drive/Colab-DeblurGANv2/031out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/032in.png"
!cp /content/Colab-DeblurGANv2/submit/032in.png "/content/drive/My Drive/Colab-DeblurGANv2/032out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/033in.png"
!cp /content/Colab-DeblurGANv2/submit/033in.png "/content/drive/My Drive/Colab-DeblurGANv2/033out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/034in.png"
!cp /content/Colab-DeblurGANv2/submit/034in.png "/content/drive/My Drive/Colab-DeblurGANv2/034out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/035in.png"
!cp /content/Colab-DeblurGANv2/submit/035in.png "/content/drive/My Drive/Colab-DeblurGANv2/035out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/036in.png"
!cp /content/Colab-DeblurGANv2/submit/036in.png "/content/drive/My Drive/Colab-DeblurGANv2/036out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/037in.png"
!cp /content/Colab-DeblurGANv2/submit/037in.png "/content/drive/My Drive/Colab-DeblurGANv2/037out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/038in.png"
!cp /content/Colab-DeblurGANv2/submit/038in.png "/content/drive/My Drive/Colab-DeblurGANv2/038out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/039in.png"
!cp /content/Colab-DeblurGANv2/submit/039in.png "/content/drive/My Drive/Colab-DeblurGANv2/039out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/040in.png"
!cp /content/Colab-DeblurGANv2/submit/040in.png "/content/drive/My Drive/Colab-DeblurGANv2/040out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/041in.png"
!cp /content/Colab-DeblurGANv2/submit/041in.png "/content/drive/My Drive/Colab-DeblurGANv2/041out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/042in.png"
!cp /content/Colab-DeblurGANv2/submit/042in.png "/content/drive/My Drive/Colab-DeblurGANv2/042out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/043in.png"
!cp /content/Colab-DeblurGANv2/submit/043in.png "/content/drive/My Drive/Colab-DeblurGANv2/043out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/044in.png"
!cp /content/Colab-DeblurGANv2/submit/044in.png "/content/drive/My Drive/Colab-DeblurGANv2/044out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/045in.png"
!cp /content/Colab-DeblurGANv2/submit/045in.png "/content/drive/My Drive/Colab-DeblurGANv2/045out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/046in.png"
!cp /content/Colab-DeblurGANv2/submit/046in.png "/content/drive/My Drive/Colab-DeblurGANv2/046out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/047in.png"
!cp /content/Colab-DeblurGANv2/submit/047in.png "/content/drive/My Drive/Colab-DeblurGANv2/047out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/048in.png"
!cp /content/Colab-DeblurGANv2/submit/048in.png "/content/drive/My Drive/Colab-DeblurGANv2/048out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/049in.png"
!cp /content/Colab-DeblurGANv2/submit/049in.png "/content/drive/My Drive/Colab-DeblurGANv2/049out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/050in.png"
!cp /content/Colab-DeblurGANv2/submit/050in.png "/content/drive/My Drive/Colab-DeblurGANv2/050out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/051in.png"
!cp /content/Colab-DeblurGANv2/submit/051in.png "/content/drive/My Drive/Colab-DeblurGANv2/051out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/052in.png"
!cp /content/Colab-DeblurGANv2/submit/052in.png "/content/drive/My Drive/Colab-DeblurGANv2/052out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/053in.png"
!cp /content/Colab-DeblurGANv2/submit/053in.png "/content/drive/My Drive/Colab-DeblurGANv2/053out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/054in.png"
!cp /content/Colab-DeblurGANv2/submit/054in.png "/content/drive/My Drive/Colab-DeblurGANv2/054out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/055in.png"
!cp /content/Colab-DeblurGANv2/submit/055in.png "/content/drive/My Drive/Colab-DeblurGANv2/055out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/056in.png"
!cp /content/Colab-DeblurGANv2/submit/056in.png "/content/drive/My Drive/Colab-DeblurGANv2/056out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/057in.png"
!cp /content/Colab-DeblurGANv2/submit/057in.png "/content/drive/My Drive/Colab-DeblurGANv2/057out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/058in.png"
!cp /content/Colab-DeblurGANv2/submit/058in.png "/content/drive/My Drive/Colab-DeblurGANv2/058out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/059in.png"
!cp /content/Colab-DeblurGANv2/submit/059in.png "/content/drive/My Drive/Colab-DeblurGANv2/059out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/060in.png"
!cp /content/Colab-DeblurGANv2/submit/060in.png "/content/drive/My Drive/Colab-DeblurGANv2/060out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/061in.png"
!cp /content/Colab-DeblurGANv2/submit/061in.png "/content/drive/My Drive/Colab-DeblurGANv2/061out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/062in.png"
!cp /content/Colab-DeblurGANv2/submit/062in.png "/content/drive/My Drive/Colab-DeblurGANv2/062out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/063in.png"
!cp /content/Colab-DeblurGANv2/submit/063in.png "/content/drive/My Drive/Colab-DeblurGANv2/063out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/064in.png"
!cp /content/Colab-DeblurGANv2/submit/064in.png "/content/drive/My Drive/Colab-DeblurGANv2/064out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/065in.png"
!cp /content/Colab-DeblurGANv2/submit/065in.png "/content/drive/My Drive/Colab-DeblurGANv2/065out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/066in.png"
!cp /content/Colab-DeblurGANv2/submit/066in.png "/content/drive/My Drive/Colab-DeblurGANv2/066out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/067in.png"
!cp /content/Colab-DeblurGANv2/submit/067in.png "/content/drive/My Drive/Colab-DeblurGANv2/067out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/068in.png"
!cp /content/Colab-DeblurGANv2/submit/068in.png "/content/drive/My Drive/Colab-DeblurGANv2/068out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/069in.png"
!cp /content/Colab-DeblurGANv2/submit/069in.png "/content/drive/My Drive/Colab-DeblurGANv2/069out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/070in.png"
!cp /content/Colab-DeblurGANv2/submit/070in.png "/content/drive/My Drive/Colab-DeblurGANv2/070out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/071in.png"
!cp /content/Colab-DeblurGANv2/submit/071in.png "/content/drive/My Drive/Colab-DeblurGANv2/071out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/072in.png"
!cp /content/Colab-DeblurGANv2/submit/072in.png "/content/drive/My Drive/Colab-DeblurGANv2/072out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/073in.png"
!cp /content/Colab-DeblurGANv2/submit/073in.png "/content/drive/My Drive/Colab-DeblurGANv2/073out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/074in.png"
!cp /content/Colab-DeblurGANv2/submit/074in.png "/content/drive/My Drive/Colab-DeblurGANv2/074out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/075in.png"
!cp /content/Colab-DeblurGANv2/submit/075in.png "/content/drive/My Drive/Colab-DeblurGANv2/075out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/076in.png"
!cp /content/Colab-DeblurGANv2/submit/076in.png "/content/drive/My Drive/Colab-DeblurGANv2/076out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/077in.png"
!cp /content/Colab-DeblurGANv2/submit/077in.png "/content/drive/My Drive/Colab-DeblurGANv2/077out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/078in.png"
!cp /content/Colab-DeblurGANv2/submit/078in.png "/content/drive/My Drive/Colab-DeblurGANv2/078out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/079in.png"
!cp /content/Colab-DeblurGANv2/submit/079in.png "/content/drive/My Drive/Colab-DeblurGANv2/079out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/080in.png"
!cp /content/Colab-DeblurGANv2/submit/080in.png "/content/drive/My Drive/Colab-DeblurGANv2/080out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/081in.png"
!cp /content/Colab-DeblurGANv2/submit/081in.png "/content/drive/My Drive/Colab-DeblurGANv2/081out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/082in.png"
!cp /content/Colab-DeblurGANv2/submit/082in.png "/content/drive/My Drive/Colab-DeblurGANv2/082out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/083in.png"
!cp /content/Colab-DeblurGANv2/submit/083in.png "/content/drive/My Drive/Colab-DeblurGANv2/083out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/084in.png"
!cp /content/Colab-DeblurGANv2/submit/084in.png "/content/drive/My Drive/Colab-DeblurGANv2/084out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/085in.png"
!cp /content/Colab-DeblurGANv2/submit/085in.png "/content/drive/My Drive/Colab-DeblurGANv2/085out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/086in.png"
!cp /content/Colab-DeblurGANv2/submit/086in.png "/content/drive/My Drive/Colab-DeblurGANv2/086out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/087in.png"
!cp /content/Colab-DeblurGANv2/submit/087in.png "/content/drive/My Drive/Colab-DeblurGANv2/087out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/088in.png"
!cp /content/Colab-DeblurGANv2/submit/088in.png "/content/drive/My Drive/Colab-DeblurGANv2/088out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/089in.png"
!cp /content/Colab-DeblurGANv2/submit/089in.png "/content/drive/My Drive/Colab-DeblurGANv2/089out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/090in.png"
!cp /content/Colab-DeblurGANv2/submit/090in.png "/content/drive/My Drive/Colab-DeblurGANv2/090out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/091in.png"
!cp /content/Colab-DeblurGANv2/submit/091in.png "/content/drive/My Drive/Colab-DeblurGANv2/091out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/092in.png"
!cp /content/Colab-DeblurGANv2/submit/092in.png "/content/drive/My Drive/Colab-DeblurGANv2/092out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/093in.png"
!cp /content/Colab-DeblurGANv2/submit/093in.png "/content/drive/My Drive/Colab-DeblurGANv2/093out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/094in.png"
!cp /content/Colab-DeblurGANv2/submit/094in.png "/content/drive/My Drive/Colab-DeblurGANv2/094out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/095in.png"
!cp /content/Colab-DeblurGANv2/submit/095in.png "/content/drive/My Drive/Colab-DeblurGANv2/095out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/096in.png"
!cp /content/Colab-DeblurGANv2/submit/096in.png "/content/drive/My Drive/Colab-DeblurGANv2/096out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/097in.png"
!cp /content/Colab-DeblurGANv2/submit/097in.png "/content/drive/My Drive/Colab-DeblurGANv2/097out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/098in.png"
!cp /content/Colab-DeblurGANv2/submit/098in.png "/content/drive/My Drive/Colab-DeblurGANv2/098out.png"
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/099in.png"
!cp /content/Colab-DeblurGANv2/submit/099in.png "/content/drive/My Drive/Colab-DeblurGANv2/099out.png"

# Deblur with MobileNet

In [ ]:
#@title Installing and downloading models
%cd /content/
!git clone https://github.com/styler00dollar/Colab-DeblurGANv2
!pip install fire
!pip install pretrainedmodels
!pip install gdown
%cd /content/Colab-DeblurGANv2/models
!gdown --id 1UXcsRVW-6KF23_TNzxw-xC0SzaMfXOaR
!wget --no-check-certificate http://sceneparsing.csail.mit.edu/model/pretrained_resnet/mobilenet_v2.pth.tar
!gdown --id 1JhnT4BBeKBBSLqTo6UsJ13HeBXevarrU

In [ ]:
#@title Changing config.yaml
%%writefile /content/Colab-DeblurGANv2/config/config.yaml
---
project: deblur_gan
experiment_desc: fpn

train:
  files_a: &FILES_A /datasets/my_dataset/**/*.jpg
  files_b: *FILES_A
  size: &SIZE 256
  crop: random
  preload: &PRELOAD false
  preload_size: &PRELOAD_SIZE 0
  bounds: [0, .9]
  scope: geometric
  corrupt: &CORRUPT
    - name: cutout
      prob: 0.5
      num_holes: 3
      max_h_size: 25
      max_w_size: 25
    - name: jpeg
      quality_lower: 70
      quality_upper: 90
    - name: motion_blur
    - name: median_blur
    - name: gamma
    - name: rgb_shift
    - name: hsv_shift
    - name: sharpen

val:
  files_a: *FILES_A
  files_b: *FILES_A
  size: *SIZE
  scope: geometric
  crop: center
  preload: *PRELOAD
  preload_size: *PRELOAD_SIZE
  bounds: [.9, 1]
  corrupt: *CORRUPT

phase: train
warmup_num: 3
model:
  g_name: fpn_mobilenet
  blocks: 9
  d_name: double_gan # may be no_gan, patch_gan, double_gan, multi_scale
  d_layers: 3
  content_loss: perceptual
  adv_lambda: 0.001
  disc_loss: wgan-gp
  learn_residual: True
  norm_layer: instance
  dropout: True

num_epochs: 200
train_batches_per_epoch: 1000
val_batches_per_epoch: 100
batch_size: 1
image_size: [256, 256]

optimizer:
  name: adam
  lr: 0.0001
scheduler:
  name: linear
  start_epoch: 50
  min_lr: 0.0000001

In [ ]:
#@title Changing predict.py
%%writefile /content/Colab-DeblurGANv2/predict.py
import os
from glob import glob
from typing import Optional

import cv2
import numpy as np
import torch
import yaml
from fire import Fire
from tqdm import tqdm

from aug import get_normalize
from models.networks import get_generator


class Predictor:
    def __init__(self, weights_path: str, model_name: str = ''):
        with open('config/config.yaml') as cfg:
            config = yaml.load(cfg)
        model = get_generator(model_name or config['model'])
        model.load_state_dict(torch.load(weights_path)['model'])
        self.model = model.cuda()
        self.model.train(True)
        # GAN inference should be in train mode to use actual stats in norm layers,
        # it's not a bug
        self.normalize_fn = get_normalize()

    @staticmethod
    def _array_to_batch(x):
        x = np.transpose(x, (2, 0, 1))
        x = np.expand_dims(x, 0)
        return torch.from_numpy(x)

    def _preprocess(self, x: np.ndarray, mask: Optional[np.ndarray]):
        x, _ = self.normalize_fn(x, x)
        if mask is None:
            mask = np.ones_like(x, dtype=np.float32)
        else:
            mask = np.round(mask.astype('float32') / 255)

        h, w, _ = x.shape
        block_size = 32
        min_height = (h // block_size + 1) * block_size
        min_width = (w // block_size + 1) * block_size

        pad_params = {'mode': 'constant',
                      'constant_values': 0,
                      'pad_width': ((0, min_height - h), (0, min_width - w), (0, 0))
                      }
        x = np.pad(x, **pad_params)
        mask = np.pad(mask, **pad_params)

        return map(self._array_to_batch, (x, mask)), h, w

    @staticmethod
    def _postprocess(x: torch.Tensor) -> np.ndarray:
        x, = x
        x = x.detach().cpu().float().numpy()
        x = (np.transpose(x, (1, 2, 0)) + 1) / 2.0 * 255.0
        return x.astype('uint8')

    def __call__(self, img: np.ndarray, mask: Optional[np.ndarray], ignore_mask=True) -> np.ndarray:
        (img, mask), h, w = self._preprocess(img, mask)
        with torch.no_grad():
            inputs = [img.cuda()]
            if not ignore_mask:
                inputs += [mask]
            pred = self.model(*inputs)
        return self._postprocess(pred)[:h, :w, :]

def process_video(pairs, predictor, output_dir):
    for video_filepath, mask in tqdm(pairs):
        video_filename = os.path.basename(video_filepath)
        output_filepath = os.path.join(output_dir, os.path.splitext(video_filename)[0]+'_deblur.mp4')
        video_in = cv2.VideoCapture(video_filepath)
        fps = video_in.get(cv2.CAP_PROP_FPS)
        width = int(video_in.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(video_in.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frame_num = int(video_in.get(cv2.CAP_PROP_FRAME_COUNT))
        video_out = cv2.VideoWriter(output_filepath, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width, height))
        tqdm.write(f'process {video_filepath} to {output_filepath}, {fps}fps, resolution: {width}x{height}')
        for frame_num in tqdm(range(total_frame_num), desc=video_filename):
            res, img = video_in.read()
            if not res:
                break
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            pred = predictor(img, mask)
            pred = cv2.cvtColor(pred, cv2.COLOR_RGB2BGR)
            video_out.write(pred)

def main(img_pattern: str,
         mask_pattern: Optional[str] = None,
         weights_path='/content/Colab-DeblurGANv2/models/fpn_mobilenet.h5',
         out_dir='submit/',
         side_by_side: bool = False,
         video: bool = False):
    def sorted_glob(pattern):
        return sorted(glob(pattern))

    imgs = sorted_glob(img_pattern)
    masks = sorted_glob(mask_pattern) if mask_pattern is not None else [None for _ in imgs]
    pairs = zip(imgs, masks)
    names = sorted([os.path.basename(x) for x in glob(img_pattern)])
    predictor = Predictor(weights_path=weights_path)

    os.makedirs(out_dir, exist_ok=True)
    if not video:
        for name, pair in tqdm(zip(names, pairs), total=len(names)):
            f_img, f_mask = pair
            img, mask = map(cv2.imread, (f_img, f_mask))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            pred = predictor(img, mask)
            if side_by_side:
                pred = np.hstack((img, pred))
            pred = cv2.cvtColor(pred, cv2.COLOR_RGB2BGR)
            cv2.imwrite(os.path.join(out_dir, name),
                        pred)
    else:
        process_video(pairs, predictor, out_dir)


if __name__ == '__main__':
    Fire(main)

In [ ]:
#@title Changing fpn_mobilenet.py
%%writefile /content/Colab-DeblurGANv2/models/fpn_mobilenet.py
import torch
import torch.nn as nn
from models.mobilenet_v2 import MobileNetV2

class FPNHead(nn.Module):
    def __init__(self, num_in, num_mid, num_out):
        super().__init__()

        self.block0 = nn.Conv2d(num_in, num_mid, kernel_size=3, padding=1, bias=False)
        self.block1 = nn.Conv2d(num_mid, num_out, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        x = nn.functional.relu(self.block0(x), inplace=True)
        x = nn.functional.relu(self.block1(x), inplace=True)
        return x


class FPNMobileNet(nn.Module):

    def __init__(self, norm_layer, output_ch=3, num_filters=64, num_filters_fpn=128, pretrained=True):
        super().__init__()

        # Feature Pyramid Network (FPN) with four feature maps of resolutions
        # 1/4, 1/8, 1/16, 1/32 and `num_filters` filters for all feature maps.

        self.fpn = FPN(num_filters=num_filters_fpn, norm_layer = norm_layer, pretrained=pretrained)

        # The segmentation heads on top of the FPN

        self.head1 = FPNHead(num_filters_fpn, num_filters, num_filters)
        self.head2 = FPNHead(num_filters_fpn, num_filters, num_filters)
        self.head3 = FPNHead(num_filters_fpn, num_filters, num_filters)
        self.head4 = FPNHead(num_filters_fpn, num_filters, num_filters)

        self.smooth = nn.Sequential(
            nn.Conv2d(4 * num_filters, num_filters, kernel_size=3, padding=1),
            norm_layer(num_filters),
            nn.ReLU(),
        )

        self.smooth2 = nn.Sequential(
            nn.Conv2d(num_filters, num_filters // 2, kernel_size=3, padding=1),
            norm_layer(num_filters // 2),
            nn.ReLU(),
        )

        self.final = nn.Conv2d(num_filters // 2, output_ch, kernel_size=3, padding=1)

    def unfreeze(self):
        self.fpn.unfreeze()

    def forward(self, x):

        map0, map1, map2, map3, map4 = self.fpn(x)

        map4 = nn.functional.upsample(self.head4(map4), scale_factor=8, mode="nearest")
        map3 = nn.functional.upsample(self.head3(map3), scale_factor=4, mode="nearest")
        map2 = nn.functional.upsample(self.head2(map2), scale_factor=2, mode="nearest")
        map1 = nn.functional.upsample(self.head1(map1), scale_factor=1, mode="nearest")

        smoothed = self.smooth(torch.cat([map4, map3, map2, map1], dim=1))
        smoothed = nn.functional.upsample(smoothed, scale_factor=2, mode="nearest")
        smoothed = self.smooth2(smoothed + map0)
        smoothed = nn.functional.upsample(smoothed, scale_factor=2, mode="nearest")

        final = self.final(smoothed)
        res = torch.tanh(final) + x

        return torch.clamp(res, min=-1, max=1)


class FPN(nn.Module):

    def __init__(self, norm_layer, num_filters=128, pretrained=True):
        """Creates an `FPN` instance for feature extraction.
        Args:
          num_filters: the number of filters in each output pyramid level
          pretrained: use ImageNet pre-trained backbone feature extractor
        """

        super().__init__()
        net = MobileNetV2(n_class=1000)

        if pretrained:
            #Load weights into the project directory
            state_dict = torch.load('/content/Colab-DeblurGANv2/models/mobilenet_v2.pth.tar') # add map_location='cpu' if no gpu
            net.load_state_dict(state_dict)
        self.features = net.features

        self.enc0 = nn.Sequential(*self.features[0:2])
        self.enc1 = nn.Sequential(*self.features[2:4])
        self.enc2 = nn.Sequential(*self.features[4:7])
        self.enc3 = nn.Sequential(*self.features[7:11])
        self.enc4 = nn.Sequential(*self.features[11:16])

        self.td1 = nn.Sequential(nn.Conv2d(num_filters, num_filters, kernel_size=3, padding=1),
                                 norm_layer(num_filters),
                                 nn.ReLU(inplace=True))
        self.td2 = nn.Sequential(nn.Conv2d(num_filters, num_filters, kernel_size=3, padding=1),
                                 norm_layer(num_filters),
                                 nn.ReLU(inplace=True))
        self.td3 = nn.Sequential(nn.Conv2d(num_filters, num_filters, kernel_size=3, padding=1),
                                 norm_layer(num_filters),
                                 nn.ReLU(inplace=True))

        self.lateral4 = nn.Conv2d(160, num_filters, kernel_size=1, bias=False)
        self.lateral3 = nn.Conv2d(64, num_filters, kernel_size=1, bias=False)
        self.lateral2 = nn.Conv2d(32, num_filters, kernel_size=1, bias=False)
        self.lateral1 = nn.Conv2d(24, num_filters, kernel_size=1, bias=False)
        self.lateral0 = nn.Conv2d(16, num_filters // 2, kernel_size=1, bias=False)

        for param in self.features.parameters():
            param.requires_grad = False

    def unfreeze(self):
        for param in self.features.parameters():
            param.requires_grad = True


    def forward(self, x):

        # Bottom-up pathway, from ResNet
        enc0 = self.enc0(x)

        enc1 = self.enc1(enc0) # 256

        enc2 = self.enc2(enc1) # 512

        enc3 = self.enc3(enc2) # 1024

        enc4 = self.enc4(enc3) # 2048

        # Lateral connections

        lateral4 = self.lateral4(enc4)
        lateral3 = self.lateral3(enc3)
        lateral2 = self.lateral2(enc2)
        lateral1 = self.lateral1(enc1)
        lateral0 = self.lateral0(enc0)

        # Top-down pathway
        map4 = lateral4
        map3 = self.td1(lateral3 + nn.functional.upsample(map4, scale_factor=2, mode="nearest"))
        map2 = self.td2(lateral2 + nn.functional.upsample(map3, scale_factor=2, mode="nearest"))
        map1 = self.td3(lateral1 + nn.functional.upsample(map2, scale_factor=2, mode="nearest"))
        return lateral0, map1, map2, map3, map4



In [ ]:
#@title Downloading model from my Google Drive (Very fast download)
!mkdir /root/
!mkdir /root/.cache/
!mkdir /root/.cache/torch/
!mkdir /root/.cache/torch/checkpoints/
%cd /root/.cache/torch/checkpoints/
!gdown --id 1y6GeaoWjhqjRjrXuZvCYEQYlblZGkE6X

In [ ]:
#@title ```[OPTIONAL / ONLY RUN THIS IF YOU HAVE PROBLEMS WITH THE PREVIOUS CELL]``` Testrun to download model file with pretrainedmodels (Download takes around 10 minutes)
%cd /content/
!wget --no-check-certificate https://raw.githubusercontent.com/xinntao/ESRGAN/master/LR/baboon.png
%cd /content/Colab-DeblurGANv2
!python predict.py /content/baboon.png

#%cd /content/DeblurGANv2/models
#!gdown --id 1JhnT4BBeKBBSLqTo6UsJ13HeBXevarrU
%cd /root/.cache/torch/checkpoints
!gdown --id 1JhnT4BBeKBBSLqTo6UsJ13HeBXevarrU
%cd /content/Colab-DeblurGANv2/models
!wget --no-check-certificate http://sceneparsing.csail.mit.edu/model/pretrained_resnet/mobilenet_v2.pth.tar

In [ ]:
#@title Run deblur and copy result go Google Drive
%cd /content/Colab-DeblurGANv2
!python predict.py "/content/drive/My Drive/Colab-DeblurGANv2/input.png"
!cp /content/Colab-DeblurGANv2/submit/input.png "/content/drive/My Drive/Colab-DeblurGANv2/output.png"